VOY A GRAFICAR LOS MAXIMOS PARA CADA GRB



In [3]:
import os, glob, csv
import numpy as np
import pandas as pd
import healpy as hp
from astropy.io import fits
from concurrent.futures import ProcessPoolExecutor
from scipy.stats import norm

# ========================= Configuración =========================
base_path  = "/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/healpixz=0_alfa1.5"
input_csv  = "/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Data/GRB_List_1.csv"
min_value  = -1e4
sig_candidates = ["SIGNIFICANCE","significance","SIG","VALUE","val","Val"]
max_workers = 20

def _read_sig_column(hdul):
    """Lee la columna de significancia o la imagen primaria."""
    if len(hdul) > 1 and hdul[1].header:
        ordering = (hdul[1].header.get("ORDERING", "RING") or "RING").strip().upper()
    else:
        ordering = (hdul[0].header.get("ORDERING", "RING") or "RING").strip().upper()
    if len(hdul) > 1 and hdul[1].data is not None:
        cols = list(hdul[1].columns.names)
        low  = [c.lower() for c in cols]
        for cand in sig_candidates:
            if cand in cols:
                return np.asarray(hdul[1].data[cand]).ravel(), ordering
            if cand.lower() in low:
                return np.asarray(hdul[1].data[cols[low.index(cand.lower())]]).ravel(), ordering
    if hdul[0].data is not None and hdul[0].data.ndim == 1:
        return np.asarray(hdul[0].data).ravel(), ordering
    raise ValueError("No significance column found")

def max_pixel_in_circle(fname, ra0, dec0, radius_deg):
    """Devuelve (valor_extremo, ra_max, dec_max) en un círculo radius_deg."""
    with fits.open(fname, memmap=True) as hdul:
        sig, ordering = _read_sig_column(hdul)
    npix = sig.size
    try:
        nside = hp.npix2nside(npix)
    except Exception:
        return np.nan, np.nan, np.nan
    vec = hp.ang2vec(np.radians(90.0 - dec0), np.radians(ra0))
    pix_ring = hp.query_disc(nside, vec, np.radians(radius_deg))
    pix_idx = hp.ring2nest(nside, pix_ring) if ordering == "NESTED" else pix_ring
    vals = sig[pix_idx]
    if min_value is not None:
        vals = vals[vals > min_value]
    if vals.size == 0:
        return np.nan, np.nan, np.nan
    # Píxel de mayor |significancia|, conservando el signo real
    i_rel = np.argmax(np.abs(vals))
    p_abs = pix_idx[i_rel]
    theta, phi = hp.pix2ang(nside, p_abs)
    return float(vals[i_rel]), float(np.degrees(phi)), float(90.0 - np.degrees(theta))

def process_seed(args):
    """Procesa un archivo seed y aplica corrección por trials (unilateral)."""
    fname, grb, transit_tag, ra0, dec0, search_r, psf = args
    seed = os.path.basename(fname).split("seed")[-1].split(".fits")[0]

    max_sig, ra_max, dec_max = max_pixel_in_circle(fname, ra0, dec0, search_r)
    if np.isnan(max_sig):
        return None

    trials_factor = (search_r / psf)**2
    # p-valor unilateral antes de trials
    p_pre = 1.0 - norm.cdf(max_sig)
    # corrección por trials
    p_post = 1.0 - (1.0 - p_pre)**trials_factor
    p_post = float(np.clip(p_post, np.finfo(float).tiny, 1 - 1e-16))
    # significancia post-trial (unilateral)
    sigma_post = norm.isf(p_post)

    return dict(
        GRB=grb,
        transit=transit_tag,
        seed=seed,
        ra_max=ra_max,
        dec_max=dec_max,
        max_sigma=max_sig,
        region_size_deg=search_r,
        trials_factor=trials_factor,
        p_pre=p_pre,
        p_post=p_post,
        sigma_post=sigma_post
    )

def run_for(psf_value: float, fixed_abs: bool):
    """Procesa todos los GRBs para una combinación de PSF y fixed_ABS."""
    print(f"\n=== Running PSF={psf_value}, fixed_ABS={fixed_abs} ===")
    coords_df = pd.read_csv(input_csv)
    tasks = []
    for _, row in coords_df.iterrows():
        grb   = row["Name"]
        ra0   = row["Ra"]
        dec0  = row["Dec"]
        fermi_err = row["Error_Radius"]
        search_r = max(psf_value, fermi_err)
        if fixed_abs:
            search_r = psf_value
        for transit_tag in ["transit_1", "transit_2"]:
            grb_path = os.path.join(base_path, grb, transit_tag)
            if not os.path.isdir(grb_path):
                continue
            pattern = os.path.join(
                grb_path,
                f"{grb}_{transit_tag}FINAL_C0.fits.gz"
            )
            for fname in glob.glob(pattern):
                tasks.append((fname, grb, transit_tag, ra0, dec0, search_r, psf_value))

    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        for res in ex.map(process_seed, tasks):
            if res:
                results.append(res)

    suffix = "fixed" if fixed_abs else "free"
    output_csv = f"Files/Max/grb_all_seed_maxima_PSF{psf_value}_{suffix}_data_negSig.csv"
    fieldnames = ["GRB","transit","seed","ra_max","dec_max","max_sigma",
                  "region_size_deg","trials_factor","p_pre","p_post","sigma_post"]
    with open(output_csv, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
    print(f"Saved {len(results)} rows to {output_csv}")

# ========================= Main loop =========================
for psf in [0.15,0.3,0.6]:
    for fixed in [True, False]:
        run_for(psf, fixed)


=== Running PSF=0.15, fixed_ABS=True ===


Saved 3 rows to Files/Max/grb_all_seed_maxima_PSF0.15_fixed_data_negSig.csv

=== Running PSF=0.15, fixed_ABS=False ===
Saved 3 rows to Files/Max/grb_all_seed_maxima_PSF0.15_free_data_negSig.csv

=== Running PSF=0.3, fixed_ABS=True ===
Saved 3 rows to Files/Max/grb_all_seed_maxima_PSF0.3_fixed_data_negSig.csv

=== Running PSF=0.3, fixed_ABS=False ===
Saved 3 rows to Files/Max/grb_all_seed_maxima_PSF0.3_free_data_negSig.csv

=== Running PSF=0.6, fixed_ABS=True ===
Saved 3 rows to Files/Max/grb_all_seed_maxima_PSF0.6_fixed_data_negSig.csv

=== Running PSF=0.6, fixed_ABS=False ===
Saved 3 rows to Files/Max/grb_all_seed_maxima_PSF0.6_free_data_negSig.csv


In [4]:
input_csv  = "/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Data/GRB_List_z=0.csv"
S=pd.read_csv(input_csv)
S[S['Error_Radius']>0.15]

,Unnamed: 0.3,Name,z+pseudo,TRIG MJD,First Transit Start MJD,First Transit Stop MJD,Tstart (GPS),Tstop (GPS),Second Transit Start MJD,Second Transit Stop MJD,...,flnc_sbpl_redfitstat,flnc_sbpl_dof,flnc_sbpl_statistic,bcatalog,scatalog,last_modified,Unnamed: 307,Trigger_time(mjd),Fermi_Name,z_wo_pseudo
5,5,GRB150811849,0.000,57245.848770,0.000000,0.000000,0.000000e+00,0.000000e+00,57246.811296,57246.982291,...,1.026,359.000,Castor C-STAT,3,3,2018-07-06 13:43:36.00,NaN,57245.848770,GRB150811849,0.0
6,6,GRB150819440,0.195,57253.439809,0.000000,0.000000,0.000000e+00,0.000000e+00,57254.394156,57254.652649,...,1.203,359.000,Castor C-STAT,3,3,2018-07-06 13:48:52.00,NaN,57253.439809,GRB150819440,0.0
7,7,GRB150922234,0.563,57287.234364,57287.977843,57288.190421,1.127000e+09,1.127018e+09,57288.975112,57289.187689,...,1.049,481.000,Castor C-STAT,3,3,2019-03-18 21:16:05.00,NaN,57287.234364,GRB150922234,0.0
14,14,GRB160806584,0.000,57606.584010,57607.028159,57607.282281,1.154566e+09,1.154588e+09,57608.025428,57608.279549,...,0.945,358.000,Castor C-STAT,3,3,2019-03-18 21:16:08.00,NaN,57606.584010,GRB160806584,0.0
16,16,GRB160822672,0.000,57622.671991,57622.985090,57623.225025,1.155944e+09,1.155965e+09,57623.982359,57624.222294,...,1.043,359.000,Castor C-STAT,3,3,2019-03-18 21:16:20.00,NaN,57622.671991,GRB160822672,0.0
17,17,GRB170206453,0.353,57790.452751,0.000000,0.000000,0.000000e+00,0.000000e+00,57791.349406,57791.609066,...,1.120,479.000,Castor C-STAT,3,3,2017-02-23 15:57:38.00,NaN,57790.452751,GRB170206453,0.0
18,18,GRB170222209,1.330,57806.209017,57806.526192,57806.794317,1.171802e+09,1.171825e+09,57807.523461,57807.791585,...,0.992,360.000,Castor C-STAT,3,3,2018-05-20 01:03:31.00,NaN,57806.209017,GRB170222209,0.0
21,21,GRB170403583,0.000,57846.582845,57847.323242,57847.587221,1.175327e+09,1.175350e+09,57848.320510,57848.584490,...,1.032,355.000,Castor C-STAT,3,3,2019-03-18 21:16:23.00,NaN,57846.582845,GRB170403583,0.0
22,22,GRB170708046,0.000,57942.045964,57942.275175,57942.540427,1.183531e+09,1.183554e+09,57943.272444,57943.537695,...,1.081,481.000,Castor C-STAT,3,4,2019-03-18 21:16:24.00,NaN,57942.045964,GRB170708046,0.0
24,24,GRB170816599,0.000,57981.599351,57982.211335,57982.469035,1.186981e+09,1.187004e+09,57983.208616,57983.466315,...,1.056,480.000,Castor C-STAT,3,4,2019-03-18 21:16:25.00,NaN,57981.599351,GRB170816599,0.0
